# Table of Contents
1. [start](#start)
2. [vars](#vars)

## Start provisioning <a name="start"></a>

In [ ]:
import time, os, random, string, sys, requests, urllib3
urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

def prov_directory(prov):
 # create working directory
 command=f"mkdir -p {prov}"
 runcmd(command)
 os.chdir(prov)

def runcmd(cmd):
    command=str(cmd).replace('\n','').replace('\r','')
    print(command)
    output=!{command}
    return(output)

def provision(hostname, domainname, url, os_variant, size, ram):
 # setup steps (also check that everything is the right format after papermill)
 hostname=str(hostname)
 domainname=str(domainname)
 url=str(url)
 os_variant=str(os_variant)
 size=str(size)
 ran=str(ram)
 domainname=domainname.split('.')[0] #primary part of domain
 f=filter(str.isalpha,hostname) #leave only alpha chars in hostname
 hostname="".join(f)
 f=filter(str.isalpha,domainname) #Leave only alpha chars in domain name
 domainname="".join(f)
 characters = string.ascii_lowercase + string.digits
 uid=''.join(random.sample(characters, 10))
 full_hostname=str(uid+"."+hostname+"."+domainname+".private") #set as .private | add uid (this is just to avoid having to chcek for collisions, bad code practice here
 # begin provisioning
 directory=str("/share/vms/disks/"+uid+"/")
 prov_directory(directory)
 image="provisionme.iso"
 open(str(directory+image), 'wb').write(requests.get(url, verify=False, headers={"user-agent": "Provision me harder"}, allow_redirects=True).content)
 command="qemu-img create -b "+directory+image+" -f qcow2 -F qcow2 "+directory+full_hostname+".img "+size
 runcmd(command)
 command="""echo "instance-id: """+uid+""" \n local-hostname: """+full_hostname+""" " > meta-data"""
 runcmd(command)
 config="""#cloud-config
hostname: {uid}.{hostname}
create_hostname_file: true
fqdn: {full_hostname}
prefer_fqdn_over_hostname: true
package_update: true
package_upgrade: false
timezone: America/Chicago
users:
  - name: ansible
    ssh_authorized_keys:
      - ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCr74xX7HTaoiY9gwjxOkmRgW/YYhz2nST4o08HYb0WuJqqk1itxP5y72EfapJhA7Juz/WQJ/8WnF5UUz5CKJJkewsOEH0ZdqTpFQFEazt/SiCdXGaeqHASv4ygHC/vWA5nJJqqs9i87wM0ItTIZfIeRubIezRpmYzuiPX4EtkPySuGa1EbvxsDKbyj59yl1KwLVGtT7Jb1iU5Fd/1BaWC/pvqJVwWjJxlc3rfP1+sRQVk2gm8lloobdSHnb1OAvIbl1dTZt4fLQ6VAJdnGJ9kA1JkpF3fUuMxZzWeVLAP3biJgyqLfDWK59Qv9X81FsLd7fAfKF5XZY7HzFUg/rJY2GAX40tNvfomYFBgQk3NOearDziYAQfCcB4uiOY93yeiNazJbfR9+e1SypLPXGcNlGvcEJdhXUoiFoDkSUNKhTKCFDpHwjNRafiGjsdwpIL5kv9jy08vtxIZU7YX6dRacm388WQE+Om5V2501cQdXkBl7NXJ8OAzcbz7Ny6qSvg0=
    sudo: ["ALL=(ALL) NOPASSWD:ALL"]
    groups: sudo
    shell: /bin/bash
packages:
  - curl
  - git
  - ansible
  - python3
runcmd:
  - curl -sSL https://bootstrap.pypa.io/get-pip.py -o /tmp/get-pip.py
  - python3 /tmp/get-pip.py --break-system-packages --root-user-action ignore
  - pip install --upgrade --break-system-packages pip pipenv
phone_home: 
    post: all
    url: 'http://10.42.0.1/?completed=$INSTANCE_ID'
""".format(uid=uid, hostname=hostname, full_hostname=full_hostname)
 # docker exec -it kali-0 msfconsole -q -x "handler -H 10.42.0.1 -P 9999 -p linux/x64/shell_reverse_tcp"
 open('user-data','w').write(config)
 command="genisoimage -output cidata.iso -V cidata -r -J user-data meta-data"
 runcmd(command)
 command="virt-install --virt-type qemu --name "+str(full_hostname)+" --ram "+str(ram)+" --disk "+str(directory)+str(full_hostname)+".img,format=qcow2 --network network=default --graphics vnc,listen=10.42.0.1 --noautoconsole --os-variant="+str(os_variant)+" --cdrom="+str(directory)+"cidata.iso --check all=off"
 runcmd(command)
 cleanup(hostname, full_hostname, domainname, url, os_variant, uid, size, ram)

def cleanup(hostname, full_hostname, domainname, url, os_variant, uid, size, ram):
 command="echo "+hostname
 runcmd(command)
 # without this sleep, sometimes dhcp assignment isn't completed
 time.sleep(300)
 command="virsh domifaddr "+full_hostname+"|grep -owE '[0-9\.]{7,20}'"
 ipaddr=str(runcmd(command))
 ipaddr=ipaddr[1:][:-1]
 print("IP ADDRESS: "+str(ipaddr))
 command="ssh-keyscan -H "+ipaddr+" >> ~/.ssh/known_hosts"
 runcmd(command)
 time.sleep(30)
 # Test ansible responsive, and validate completion
 command="""ansible all -i """+ipaddr+""", -a "cloud-init status --wait" -u ansible -e 'wait=true' """
 runcmd(command)


## Set variables <a name="vars"></a>

In [ ]:
hostname="ubuntu"
domainname="prov"
# tmux new-session -d 'python3 -m http.server --directory /share/vms/isos/ 8000'
url="http://10.42.0.1:8000/noble-server-cloudimg-amd64.img"
os_variant="ubuntu-stable-latest"
size="80G"
ram="2096"
# need to run AFTER this cell, because papermill injects after this, not in the middle of it.

In [ ]:
provision(hostname, domainname, url, os_variant, size, ram)